In [73]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
import math
import datetime as dt

In [74]:
#cleaned = pd.read_csv('/Users/jordanlabuda/Documents/GitHub/sosc133/CleanSOSC.csv')
cleaned = pd.read_csv('CleanSOSC.csv')

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [75]:
cleaned

,Unnamed: 0,gvkey,splticrm,datadate,tic,cusip_x,date,NCUSIP,TICKER,PERMNO,COMNAM,cusip_y,NWPERM,PRC,VOL,RET
0,0,1003,NaN,20100131,ANTQ,0003541,2010-01-31,000354100,ANTQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1003,NaN,20100228,ANTQ,0003541,2010-02-28,000354100,ANTQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1003,NaN,20100331,ANTQ,0003541,2010-03-31,000354100,ANTQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1003,NaN,20100430,ANTQ,0003541,2010-04-30,000354100,ANTQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1003,NaN,20100531,ANTQ,0003541,2010-05-31,000354100,ANTQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
832964,832964,316056,BBB-,20161031,ALLE,G0176J1,2016-10-31,G0176J109,ALLE,14297.0,ALLEGION PLC,G0176J10,NaN,71.22,573928.0,-0.000561
832965,832965,316056,BBB-,20161130,ALLE,G0176J1,2016-11-30,G0176J109,ALLE,14297.0,ALLEGION PLC,G0176J10,NaN,68.91,1014660.0,0.009522
832966,832966,316056,BBB-,20161231,ALLE,G0176J1,2016-12-31,G0176J109,ALLE,14297.0,ALLEGION PLC,G0176J10,NaN,63.84,1601109.0,0.003931
832967,832967,316056,BBB-,20170131,ALLE,G0176J1,2017-01-31,G0176J109,ALLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
cleaned.columns

Index(['Unnamed: 0', 'gvkey', 'splticrm', 'datadate', 'tic', 'cusip_x', 'date',
       'NCUSIP', 'TICKER', 'PERMNO', 'COMNAM', 'cusip_y', 'NWPERM', 'PRC',
       'VOL', 'RET'],
      dtype='object')

Below is correct for what you wanted to do, but its a void method so we need to put cleaned= before it. Lots of pandas methods have a parameter called inplace which allows you to not have to do this.

In [77]:
#idk how to specify if only price is null but seems related so just gonna drop them all
cleaned = cleaned.dropna(subset=['PRC', 'splticrm'])

In [78]:
#binned = cleaned.groupby('splticrm')

In [79]:
#binned.groups

In [80]:
binsdict = {
    'D': 1,
    'SD': 2,
    'RD': 3,
    'C': 4,
    'CC': 5,
    'CCC-': 6,
    'CCC': 7,
    'CCC+': 8,
    'B-': 9,
    'B': 10,
    'B+' : 11,
    'BB-': 12,
    'BB': 13,
    'BB+': 14,
    'BBB-': 15,
    'BBB': 16,
    'BBB+' : 17,
    'A-': 18,
    'A': 19,
    'A+': 20,
    'AA-': 21,
    'AA': 22,
    'AA+': 23,
    'AAA': 24
}


In [81]:
#this is a pretty and generalizable way to apply lambdas to a dataframe
func = lambda x: binsdict[x]
cleaned['num_cr'] = cleaned['splticrm'].apply(func)

<ipython-input-81-f7a6904ae199>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned['num_cr'] = cleaned['splticrm'].apply(func)


In [82]:
cleaned

,Unnamed: 0,gvkey,splticrm,datadate,tic,cusip_x,date,NCUSIP,TICKER,PERMNO,COMNAM,cusip_y,NWPERM,PRC,VOL,RET,num_cr
39,39,1004,BB,20100531,AIR,0003611,2010-05-31,000361105,AIR,54594.0,A A R CORP,00036110,NaN,24.82,229900.0,-0.010761,13
40,40,1004,BB,20100630,AIR,0003611,2010-06-30,000361105,AIR,54594.0,A A R CORP,00036110,NaN,24.38,184900.0,-0.034455,13
44,44,1004,BB,20101031,AIR,0003611,2010-10-31,000361105,AIR,54594.0,A A R CORP,00036110,NaN,15.36,278400.0,-0.015385,13
47,47,1004,BB,20110131,AIR,0003611,2011-01-31,000361105,AIR,54594.0,A A R CORP,00036110,NaN,24.56,308200.0,0.0107,13
48,48,1004,BB,20110228,AIR,0003611,2011-02-28,000361105,AIR,54594.0,A A R CORP,00036110,NaN,27.47,257200.0,0.004755,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
832958,832958,316056,BB+,20160430,ALLE,G0176J1,2016-04-30,G0176J109,ALLE,14297.0,ALLEGION PLC,G0176J10,NaN,63.00,1127875.0,-0.013776,14
832961,832961,316056,BBB-,20160731,ALLE,G0176J1,2016-07-31,G0176J109,ALLE,14297.0,ALLEGION PLC,G0176J10,NaN,67.64,563701.0,-0.005148,15
832964,832964,316056,BBB-,20161031,ALLE,G0176J1,2016-10-31,G0176J109,ALLE,14297.0,ALLEGION PLC,G0176J10,NaN,71.22,573928.0,-0.000561,15
832965,832965,316056,BBB-,20161130,ALLE,G0176J1,2016-11-30,G0176J109,ALLE,14297.0,ALLEGION PLC,G0176J10,NaN,68.91,1014660.0,0.009522,15


In [101]:
#tics=cleaned.groupby('tic')

In [102]:
#tics['our_ret']=tics['PRC'].Series.GroupBy.apply(pct_change)
cleaned['our_ret']=cleaned.groupby('tic')['PRC'].apply(lambda x: x.pct_change())
cleaned['log_our_ret']=cleaned.groupby('tic')['PRC'].apply(lambda x: np.log(x).pct_change())

<ipython-input-102-e5cc2aad3e7c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned['our_ret']=cleaned.groupby('tic')['PRC'].apply(lambda x: x.pct_change())
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
<ipython-input-102-e5cc2aad3e7c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned['log_our_ret']=cleaned.groupby('tic')['PRC'].apply(lambda x: np.log(x).pct_change())

In [103]:
cleaned

,Unnamed: 0,gvkey,splticrm,datadate,tic,cusip_x,date,NCUSIP,TICKER,PERMNO,COMNAM,cusip_y,NWPERM,PRC,VOL,RET,num_cr,our_ret,log_our_ret
56,56,1004,BB,20111031,AIR,0003611,2011-10-31,000361105,AIR,54594.0,A A R CORP,00036110,NaN,23.59,282300.0,-0.002537,13,NaN,NaN
57,57,1004,BB,20111130,AIR,0003611,2011-11-30,000361105,AIR,54594.0,A A R CORP,00036110,NaN,16.67,359200.0,-0.043603,13,-0.293345,-0.109849
58,58,1004,BB,20111231,AIR,0003611,2011-12-31,000361105,AIR,54594.0,A A R CORP,00036110,NaN,19.93,277900.0,-0.02208,13,0.195561,0.063483
61,61,1004,BB,20120331,AIR,0003611,2012-03-31,000361105,AIR,54594.0,A A R CORP,00036110,NaN,21.19,162700.0,0.013391,13,0.063221,0.020488
64,64,1004,BB,20120630,AIR,0003611,2012-06-30,000361105,AIR,54594.0,A A R CORP,00036110,NaN,15.45,260000.0,-0.003868,13,-0.270882,-0.103461
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
832958,832958,316056,BB+,20160430,ALLE,G0176J1,2016-04-30,G0176J109,ALLE,14297.0,ALLEGION PLC,G0176J10,NaN,63.00,1127875.0,-0.013776,14,-0.062639,-0.015373
832961,832961,316056,BBB-,20160731,ALLE,G0176J1,2016-07-31,G0176J109,ALLE,14297.0,ALLEGION PLC,G0176J10,NaN,67.64,563701.0,-0.005148,15,0.073651,0.017152
832964,832964,316056,BBB-,20161031,ALLE,G0176J1,2016-10-31,G0176J109,ALLE,14297.0,ALLEGION PLC,G0176J10,NaN,71.22,573928.0,-0.000561,15,0.052927,0.012238
832965,832965,316056,BBB-,20161130,ALLE,G0176J1,2016-11-30,G0176J109,ALLE,14297.0,ALLEGION PLC,G0176J10,NaN,68.91,1014660.0,0.009522,15,-0.032435,-0.007730


In [104]:
cleaned = cleaned.dropna(subset=['our_ret','log_our_ret'])

In [105]:
cleaned

,Unnamed: 0,gvkey,splticrm,datadate,tic,cusip_x,date,NCUSIP,TICKER,PERMNO,COMNAM,cusip_y,NWPERM,PRC,VOL,RET,num_cr,our_ret,log_our_ret
57,57,1004,BB,20111130,AIR,0003611,2011-11-30,000361105,AIR,54594.0,A A R CORP,00036110,NaN,16.67,359200.0,-0.043603,13,-0.293345,-0.109849
58,58,1004,BB,20111231,AIR,0003611,2011-12-31,000361105,AIR,54594.0,A A R CORP,00036110,NaN,19.93,277900.0,-0.02208,13,0.195561,0.063483
61,61,1004,BB,20120331,AIR,0003611,2012-03-31,000361105,AIR,54594.0,A A R CORP,00036110,NaN,21.19,162700.0,0.013391,13,0.063221,0.020488
64,64,1004,BB,20120630,AIR,0003611,2012-06-30,000361105,AIR,54594.0,A A R CORP,00036110,NaN,15.45,260000.0,-0.003868,13,-0.270882,-0.103461
67,67,1004,BB,20120930,AIR,0003611,2012-09-30,000361105,AIR,54594.0,A A R CORP,00036110,NaN,14.21,354500.0,0.007087,13,-0.080259,-0.030561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
832958,832958,316056,BB+,20160430,ALLE,G0176J1,2016-04-30,G0176J109,ALLE,14297.0,ALLEGION PLC,G0176J10,NaN,63.00,1127875.0,-0.013776,14,-0.062639,-0.015373
832961,832961,316056,BBB-,20160731,ALLE,G0176J1,2016-07-31,G0176J109,ALLE,14297.0,ALLEGION PLC,G0176J10,NaN,67.64,563701.0,-0.005148,15,0.073651,0.017152
832964,832964,316056,BBB-,20161031,ALLE,G0176J1,2016-10-31,G0176J109,ALLE,14297.0,ALLEGION PLC,G0176J10,NaN,71.22,573928.0,-0.000561,15,0.052927,0.012238
832965,832965,316056,BBB-,20161130,ALLE,G0176J1,2016-11-30,G0176J109,ALLE,14297.0,ALLEGION PLC,G0176J10,NaN,68.91,1014660.0,0.009522,15,-0.032435,-0.007730


In [106]:
model=sm.OLS(cleaned['our_ret'],cleaned['num_cr'])
logmodel=sm.OLS(cleaned['log_our_ret'],cleaned['num_cr'])
result=model.fit()
logresult=logmodel.fit()
print(result.summary())
print(logresult.summary())

                                 OLS Regression Results                                
Dep. Variable:                our_ret   R-squared (uncentered):                   0.002
Model:                            OLS   Adj. R-squared (uncentered):              0.002
Method:                 Least Squares   F-statistic:                              64.11
Date:                Wed, 26 May 2021   Prob (F-statistic):                    1.22e-15
Time:                        00:53:08   Log-Likelihood:                         -9447.8
No. Observations:               28825   AIC:                                  1.890e+04
Df Residuals:                   28824   BIC:                                  1.891e+04
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------